In [1]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from chromalab.observer import Observer, Cone, transformToChromaticity, getHeringMatrix
from chromalab.spectra import Spectra, Illuminant, convert_refs_to_spectras
from chromalab.maxbasis import MaxBasis
from chromalab.visualizer import PSWrapper, DisplayBasisType, exportAndPlay
from chromalab.spectral_analysis import PCAAnalysis


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [4]:
wavelengths1 = np.arange(400, 701, 1)
wavelengths5 = np.arange(400, 701, 5)
wavelengths10 = np.arange(400, 701, 10)

In [5]:
wavelengths = np.arange(400, 710, 10)
d65 = Illuminant.get("D65")

In [6]:
trichromat = Observer.trichromat(wavelengths=wavelengths, illuminant=d65) 
maxbasis_tri = MaxBasis(trichromat, verbose=True)
LMS_to_RGB = maxbasis_tri.get_cone_to_maxbasis_transform()[::-1]

100%|██████████| 28/28 [00:00<00:00, 72.49it/s]
/home/jlee/ChromaLab/chromalab/spectra.py:36: UserWarning: Data has values not between 0 and 1. Clipping.
  warnings.warn("Data has values not between 0 and 1. Clipping.")


In [ ]:
# natural_spectra = np.load("../../../Hyperspectral-Data/natural-spectra-400-700-5.npy")
lmsq_data = np.load("./data/lmsq.npy").reshape(-1, 4)

tetrachromat = Observer.tetrachromat(wavelengths=wavelengths, illuminant=d65, verbose=True) 
maxbasis = MaxBasis(tetrachromat, verbose=True)

viz = PSWrapper(tetrachromat, maxbasis, itemsToDisplay=PSWrapper.ItemsToDisplay.MESH, displayBasis=DisplayBasisType.MAXBASIS, verbose=True)

In [7]:
pca = PCAAnalysis(tetrachromat)
lmsq = pca.get_LMSQ_Responses()
np.save("lmsq.npy", lmsq)

100%|██████████| 9/9 [00:02<00:00,  3.02it/s]


In [ ]:
T = maxbasis.get_cone_to_maxbasis_transform()
responses = (viz.HMatrix@T@lmsq_data.T).T[:, 1:]
rgbs = np.clip((LMS_to_RGB@(lmsq_data[:, [0, 1, 3]].T)).T, 0, 1)

In [ ]:
viz.ps.set_ground_plane_mode("none")
viz.renderObjectsPS(mesh_alpha=0.4, lattice_alpha=0.1)
viz.ps.get_surface_mesh("mesh").set_transform(viz._getTransformQUpDir())
viz.renderPointCloud(responses[::100], rgbs[::100], radius=0.01)
viz.ps.get_point_cloud("points").set_transform(viz._getTransformQUpDir())
viz.renderQArrow(radius=0.003)
viz.ps.get_surface_mesh("qarrow").set_transform(viz._getTransformQUpDir())

In [ ]:
# viz.ps.show()

In [ ]:
dirname = "./outputs/pca-point-cloud"
os.makedirs(dirname, exist_ok=True)
def rotate_once(offset, frame_count, theta):
    for j in range(frame_count): # rotate once
        phi = 360 * j / frame_count
        point_3d = PSWrapper.polarToCartesian(3, theta, phi)
        viz.ps.look_at(point_3d, [0, 0, 0])
        viz.ps.screenshot(dirname + f"/frame_{offset * frame_count + j:03d}.png", True)

def up_down_once(offset, frame_count, start_theta):
    thetas = np.cos(np.linspace(np.radians(start_theta), 2*np.pi + np.radians(start_theta), frame_count)) * 45 + 45
    # plt.plot(thetas)
    # plt.show()
    for j in range(frame_count): # rotate once
        # theta = 90 * (start_theta + j) / frame_count
        point_3d = PSWrapper.polarToCartesian(3, thetas[j], 0)
        viz.ps.look_at(point_3d, [0, 0, 0])
        viz.ps.screenshot(dirname + f"/frame_{offset * frame_count + j:03d}.png", True)
    
rot_per_sec = 0.3
frame_count = int(1/rot_per_sec * 30)
start_theta = 70

rotate_once(0, frame_count, start_theta)
up_down_once(1, frame_count, start_theta)

In [ ]:
exportAndPlay(dirname)